# Crystal relaxation and elastic constants

This Notebook generates

**Library imports**

In [47]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)

# http://www.numpy.org/
import numpy as np

from IPython.core.display import display, HTML

# https://pandas.pydata.org/
import pandas as pd

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.a


## 0. Access database 

### Load database

In [2]:
database = iprPy.load_database('master')

### Copy references from library, if needed

In [3]:
database.build_refs()
for style in ['crystal_prototype', 'dislocation_monopole', 'free_surface', 'point_defect', 'potential_LAMMPS', 'stacking_fault']:
    database.check_records(style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 19 of style crystal_prototype
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 5 of style dislocation_monopole
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 6 of style free_surface
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 38 of style point_defect
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 184 of style potential_LAMMPS
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 3 of style stacking_fault


## 1. E_vs_r_scan calculation

In [4]:
calculation = iprPy.load_calculation('E_vs_r_scan')
run_directory = iprPy.load_run_directory('master_1')

### Write input script

In [5]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on prototype records
buildcombos                 crystalprototype load_file prototype
prototype_potential_name    2004--Zhou-X-W--Al--LAMMPS--ipr2

# System manipulations
x_axis                      
y_axis                      
z_axis                      
atomshift                   
sizemults                   3 3 3

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
minimum_r                   2.0
maximum_r                   6.0
number_of_steps_r           200  
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [6]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 10 of style calculation_E_vs_r_scan
 - 10 are complete
 - 0 still to run
 - 0 issued errors


### Run calculations

In [7]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 11172
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 10 of style calculation_E_vs_r_scan
 - 10 are complete
 - 0 still to run
 - 0 issued errors


## 2. relax_box calculation

In [8]:
calculation = iprPy.load_calculation('relax_box')
run_directory = iprPy.load_run_directory('master_1')

### Write input script

In [9]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference
reference_potential_name    2004--Zhou-X-W--Al--LAMMPS--ipr2

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# System manipulations
x_axis                      
y_axis                      
z_axis                      
atomshift                   
sizemults                   3 3 3

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
strainrange                 1e-6
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [10]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 13 of style calculation_relax_box
 - 11 are complete
 - 0 still to run
 - 2 issued errors


### Run calculations

In [11]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 11172
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 13 of style calculation_relax_box
 - 11 are complete
 - 0 still to run
 - 2 issued errors


In [12]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
  File "calc_relax_box.py", line 455, in <module>
    main(*sys.argv[1:])
  File "calc_relax_box.py", line 54, in main
    strainrange = input_dict[\'strainrange\'])
  File "calc_relax_box.py", line 145, in relax_box
    strainrange=strainrange, cycle=cycle)
  File "calc_relax_box.py", line 364, in calc_cij
    raise RuntimeError(\'Divergence of box dimensions to <= 0\')
RuntimeError: Divergence of box dimensions to <= 0
'


## 3. relax_dynamic calculation

In [13]:
calculation = iprPy.load_calculation('relax_dynamic')
run_directory = iprPy.load_run_directory('master_4')

### Write input script

In [14]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 mpiexec -localonly 4

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference
reference_potential_name    2004--Zhou-X-W--Al--LAMMPS--ipr2

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# System manipulations
x_axis                      
y_axis                      
z_axis                      
atomshift                   
sizemults                   10 10 10

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
temperature                 0.0
pressure_xx                 
pressure_yy                 
pressure_zz                 
pressure_xy                 
pressure_xz                 
pressure_yz                 
integrator                  nph+l
thermosteps                 1000
dumpsteps                   
runsteps                    10000
equilsteps                  0
randomseed                  
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [15]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 13 of style calculation_relax_dynamic
 - 13 are complete
 - 0 still to run
 - 0 issued errors


### Run calculations

In [16]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 11172
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 13 of style calculation_relax_dynamic
 - 13 are complete
 - 0 still to run
 - 0 issued errors


In [17]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

## 4. relax_static calculation

In [18]:
calculation = iprPy.load_calculation('relax_static')
run_directory = iprPy.load_run_directory('master_1')

### Write input script

In [19]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference
reference_potential_name    2004--Zhou-X-W--Al--LAMMPS--ipr2

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# Build load information from relax_dynamic results
buildcombos                 atomicarchive load_file archive
archive_record              calculation_relax_dynamic
archive_load_key            final-system

# System manipulations
x_axis                      
y_axis                      
z_axis                      
atomshift                   
sizemults                   1 1 1

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
energytolerance             0.0
forcetolerance              1e-10 eV/angstrom
maxiterations               10000
maxevaluations              100000
maxatommotion               0.01 angstrom
maxcycles                   100
cycletolerance              1e-10
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [20]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 26 of style calculation_relax_static
 - 25 are complete
 - 0 still to run
 - 1 issued errors


### Run calculations

In [21]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 11172
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 26 of style calculation_relax_static
 - 25 are complete
 - 0 still to run
 - 1 issued errors


In [22]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
  File "calc_relax_static.py", line 369, in <module>
    main(*sys.argv[1:])
  File "calc_relax_static.py", line 43, in main
    process_input(input_dict, *args[1:])
  File "calc_relax_static.py", line 366, in process_input
    iprPy.input.interpret(\'atomman_systemmanipulate\', input_dict, build=build)
  File "c:\\users\\lmh1\\documents\\python-packages\\iprpy\\iprPy\\input\\interpret.py", line 14, in interpret
    loaded[style](input_dict, build=build, **kwargs)
  File "c:\\users\\lmh1\\documents\\python-packages\\iprpy\\iprPy\\input\\interpret_functions\\atomman_systemmanipulate.py", line 136, in atomman_systemmanipulate
    initialsystem = ucell.rotate(axes)
  File "c:\\users\\lmh1\\documents\\python-packages\\atomman\\atomman\\core\\System.py", line 661, in rotate
    \'atoms expected, \' + str(len(aindex[0])) + \' found\')
ValueError: Filtering failed: 2000atoms expected, 1729 found
'


## 5. crystal_space_group calculation

In [23]:
calculation = iprPy.load_calculation('crystal_space_group')
run_directory = iprPy.load_run_directory('master_1')

### Write input script

In [24]:
input_script = """

# Build load information based on prototype records
buildcombos                 crystalprototype load_file

# Build load information based on reference structures
buildcombos                 atomicreference load_file ref
ref_elements                Al

# Build load information from relax_static results
buildcombos                 atomicarchive load_file relax_static
relax_static_record         calculation_relax_static
relax_static_load_key       final-system

# Build load information from relax_box results
buildcombos                 atomicarchive load_file relax_box
relax_box_record            calculation_relax_box
relax_box_load_key          final-system

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
symmetryprecision           
primitivecell               
idealcell                   
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [25]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 58 of style calculation_crystal_space_group
 - 0 are complete
 - 58 still to run
 - 0 issued errors


### Run calculations

In [26]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 11172
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 58 of style calculation_crystal_space_group
 - 58 are complete
 - 0 still to run
 - 0 issued errors


In [27]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

## 6. Analyze

### Get parent records

In [36]:
relax_box_df = database.get_records_df(style='calculation_relax_box', full=True, flat=True)
relax_static_df = database.get_records_df(style='calculation_relax_static', full=True, flat=True)
relax_dynamic_df = database.get_records_df(style='calculation_relax_dynamic', full=True, flat=True)
crystal_space_group_df = database.get_records_df(style='calculation_crystal_space_group', full=True, flat=True) 

### Split into reference, prototype and results

In [29]:
reference_df = crystal_space_group_df[(crystal_space_group_df.family+'.poscar'==crystal_space_group_df.load_file)]
prototype_df = crystal_space_group_df[(crystal_space_group_df.family+'.json'==crystal_space_group_df.load_file)]
reference_index = reference_df.index
prototype_index = prototype_df.index
relaxed_index = crystal_space_group_df.index.difference(reference_index.union(prototype_index))
relaxed_df = crystal_space_group_df.iloc[relaxed_index]

reference_df = reference_df.reset_index()
prototype_df = prototype_df.reset_index()
relaxed_df = relaxed_df.reset_index()

### Indentify structures that transformed

In [30]:
match_df = pd.read_csv('reference_prototype_match.csv')

In [31]:
results_df = []
for series in relaxed_df.itertuples():
    results_dict = {}
    results_dict['key'] = series.key
    results_dict['family'] = series.family
    results_dict['a (Å)'] = series.a
    results_dict['b (Å)'] = series.b
    results_dict['c (Å)'] = series.c
    results_dict['alpha'] = series.alpha
    results_dict['beta'] = series.beta
    results_dict['gamma'] = series.gamma
    
    try:
        family_series = reference_df[reference_df.family == series.family].iloc[0]
    except:
        family_series = prototype_df[prototype_df.family == series.family].iloc[0]
        results_dict['prototype'] = series.family
    else:
        prototype = match_df[match_df.reference==series.family].prototype.values[0]
        if prototype != 'multiple':
            results_dict['prototype'] = prototype
    
    results_dict['transformed'] = not (family_series.spacegroup_number == series.spacegroup_number
                                       and family_series.pearson_symbol == series.pearson_symbol)
    
    for parent in database.get_parent_records(name=series.key):
        parent_dict = parent.todict()
        if parent_dict['key'] in relax_box_df.key.tolist():
            results_dict['method'] = 'box'
            results_dict['E_cohesive (eV)'] = parent_dict['E_cohesive']
            results_dict['potential_LAMMPS_id'] = parent_dict['potential_LAMMPS_id']
            continue
        elif parent_dict['key'] in relax_dynamic_df.key.tolist():
            results_dict['method'] = 'dynamic'
        elif parent_dict['key'] in relax_static_df.key.tolist():
            if 'relaxation' not in results_dict:
                results_dict['method'] = 'static'
            results_dict['E_cohesive (eV)'] = parent_dict['E_cohesive']
            results_dict['potential_LAMMPS_id'] = parent_dict['potential_LAMMPS_id']
    
    results_df.append(results_dict)
results_df = pd.DataFrame(results_df).sort_values('E_cohesive (eV)')

### Filter out transformed and duplicate structures

In [32]:
filtered_df = results_df[results_df.transformed == False].reset_index()

skipindex = set()
nresults = len(filtered_df)
for i in range(nresults):
    iseries = filtered_df.iloc[i]
    for j in range(i+1, nresults):
        jseries = filtered_df.iloc[j]
        if (np.isclose(iseries['E_cohesive (eV)'], jseries['E_cohesive (eV)'], rtol=0.0, atol=0.001)
            and np.isclose(iseries['a (Å)'], jseries['a (Å)'], rtol=0.0, atol=0.001)
            and np.isclose(iseries['b (Å)'], jseries['b (Å)'], rtol=0.0, atol=0.001)
            and np.isclose(iseries['c (Å)'], jseries['c (Å)'], rtol=0.0, atol=0.001)):
            
            if iseries.method == 'dynamic' or iseries.method == jseries.method:
                skipindex.add(j)
            elif jseries.method == 'dynamic':
                skipindex.add(i)
            elif iseries.method == 'static':
                skipindex.add(j)
            elif jseries.method == 'static':
                skipindex.add(i)
            else:
                raise ValueError('oops!')
            
filtered_df = filtered_df.iloc[filtered_df.index.difference(skipindex)]

In [33]:
show_df = filtered_df[filtered_df.potential_LAMMPS_id == '2004--Zhou-X-W--Al--LAMMPS--ipr2']
show_df = show_df[['prototype', 'method', 'E_cohesive (eV)', 'a (Å)', 'b (Å)', 'c (Å)', 'alpha', 'beta', 'gamma']]
display(HTML(show_df.to_html(index=False)))

prototype,method,E_cohesive (eV),a (Å),b (Å),c (Å),alpha,beta,gamma
A1--Cu--fcc,dynamic,-3.580002,4.050200,4.050200,4.050200,90.0,90.0,90.0
A3--Mg--hcp,dynamic,-3.578845,2.831200,2.831200,4.893291,90.0,90.0,120.0
A3'--alpha-La--double-hcp,dynamic,-3.577534,2.839117,2.839117,9.658255,90.0,90.0,120.0
A15--beta-W,dynamic,-3.563638,5.173408,5.173408,5.173408,90.0,90.0,90.0
A2--W--bcc,static,-3.546300,3.309992,3.309992,3.309992,90.0,90.0,90.0
A5--beta-Sn,static,-3.465649,5.273533,5.273533,2.807452,90.0,90.0,90.0
Ah--alpha-Po--sc,static,-3.435405,2.716499,2.716499,2.716499,90.0,90.0,90.0
A4--C--dc,static,-3.025113,5.662937,5.662937,5.662937,90.0,90.0,90.0


## 7. elastic_constants_static calculation

In [34]:
calculation = iprPy.load_calculation('elastic_constants_static')
run_directory = iprPy.load_run_directory('master_1')

### Manually prepare calculation

In [35]:
database.get_records(name=filtered_df.key)

In [64]:
input_dict = {}
input_dict['lammps_command'] = 'lmp_mpi'
input_dict['strainrange'] = ['1e-5', '1e-6', '1e-7', '1e-8']
input_dict['sizemults'] = '3 3 3'
records, record_df = database.get_records(name=filtered_df.key, return_df=True)
    
keys = ['potential_file','potential_content','potential_dir', 'load_file','load_content', 'load_style',
        'family','load_options','symbols', 'box_parameters']
for key in keys:
    input_dict[key] = []

for i, record_info in record_df.iterrows():
    record = records[i]
    
    for parent in database.get_parent_records(record=record):
        parent_dict = parent.todict()
        if 'potential_LAMMPS_id' in parent_dict:
            potential = database.get_record(name=parent_dict['potential_LAMMPS_id'])
            break
    
    input_dict['potential_file'].append(potential.name + '.json')
    input_dict['potential_content'].append(potential.content.json(indent=4))
    input_dict['potential_dir'].append(potential.name)
    input_dict['load_file'].append(record.name + '.json')
    input_dict['load_content'].append(record.content.json(indent=4))
    input_dict['load_style'].append('system_model')
    input_dict['family'].append('')
    input_dict['load_options'].append('key unit-cell-atomic-system')
    input_dict['symbols'].append('')
    input_dict['box_parameters'].append('')

In [65]:
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 32 of style calculation_elastic_constants_static
 - 8 are complete
 - 24 still to run
 - 0 issued errors


In [66]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 11172
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 32 of style calculation_elastic_constants_static
 - 32 are complete
 - 0 still to run
 - 0 issued errors


In [76]:
elastic_df = database.get_records_df(style=calculation.record_style)

for elastic_series in elastic_df[elastic_df.family=="A1--Cu--fcc"].itertuples():
    print('%s %7.2f %7.2f %7.2f' % (elastic_series.strainrange,
                                    uc.get_in_units(elastic_series.C.Cij[0,0], 'GPa'),
                                    uc.get_in_units(elastic_series.C.Cij[0,1], 'GPa'),
                                    uc.get_in_units(elastic_series.C.Cij[3,3], 'GPa')))

1e-05  -30.73   -5.51  -14.72
1e-08  106.99   60.48   28.30
1e-07  106.99   60.48   28.30
1e-06 -1154.93   17.31 -401.84


In [52]:
database.destroy_records('calculation_elastic_constants_static')

8 records found for calculation_elastic_constants_static
Delete records? (must type yes): yes
8 records successfully deleted
